# Parallel and Distributed Training of Deep Learning Models

## Need for parallel and distributed training:
The need for parallel and distributed algorithms in deep learning arises due to the intricate nature of neural networks, encompassing millions of parameters that necessitate vast amounts of data for effective learning. Training these models is a computationally demanding and time-consuming task. For instance, training a deep neural network like VGGNet on a single core CPU typically takes several days, highlighting the intensive nature of this process. Even when employing CPUs with multiple cores say $n$, the training time reduces to approximately $1/n$ of the original duration, which still translates to several hours, indicating the substantial time investment involved. Additionally, the sheer size of datasets often exceeds the storage capacity of a single machine. Therefore, the development of parallel and distributed algorithms becomes crucial to expedite training procedures and notably diminish the time required for these intricate deep learning tasks.

## Parallel and Distributed Methods

Some of the more common methods to parallelize and/or distribute computation across multiple machines and multiple cores are as follow:

**1) Local training:** The model and data is stored on a single machine.
- Multi-core processing
- Use GPU
- Use bth multi-core processing and GPU

**2) Distributed training:** Data or model are stored across multiple machines when it is not possible to store entire dataset or model on a single machine.
- Data parallelism
- Model parallelism

## 1) Data parallelism


In data parallelism, the training dataset undergoes division into multiple subsets, each processed on distinct GPUs (worker nodes) hosting replicated models. To maintain consistency in training, these subsets synchronize the model parameters (known as "gradients") after computing each batch. This synchronization is crucial as individual devices independently calculate errors based on their predictions and corresponding labeled outputs for their respective training samples. Hence, alterations made by each device need to be communicated to all models across all devices.

An intriguing characteristic of this approach is its scalability proportional to the dataset size, accelerating how quickly the entire dataset contributes to optimization. Moreover, it minimizes inter-node communication due to a higher computation-per-weight ratio. However, it necessitates accommodating the entire model on each node and finds its primary utility in expediting computations for convolutional neural networks handling large datasets.

<!-- ![data_parallelism](https://drive.google.com/uc?id=1GZzeJA0gzb0PV3H0VRB4h3abSsUOzloo) -->
![data_parallelism](https://i.postimg.cc/yNMY43D8/image.png)

### Concurrency in Training via Data Parallelism

Within distributed settings, multiple instances of Stochastic Gradient Descent (SGD) may operate independently. The challenge arises in how to combine these multiple gradients effectively.

**Synchronous and Asynchronous Distributed Training:**

Stochastic Gradient Descent (SGD) operates iteratively, encompassing several rounds of training wherein each round's outcomes contribute to the model's preparation for subsequent rounds. These rounds can be executed on multiple devices, employing either synchronous or asynchronous methods.

Each iteration of SGD involves processing a mini-batch of training samples. In synchronous training, all devices train their local models using different data segments from a single, typically large, mini-batch. Subsequently, they exchange their locally computed gradients, directly or indirectly, with all devices. Only after all devices have completed gradient computation and transmitted their updates does the model get updated. The revised model is then disseminated to all nodes alongside subdivisions from the subsequent mini-batch.

In contrast, asynchronous training operates without devices waiting for model updates from one another. Devices can function independently, sharing outcomes as peers or communicating through one or more central servers, often referred to as "parameter" servers. Within the peer architecture, each device runs a continuous loop involving data reading, gradient computation, direct or indirect gradient sharing with all devices, and updating the model to the most recent version.

Practical implementations tend to adopt synchronous approaches for clusters of up to 32–50 nodes, while opting for asynchronous methodologies for larger clusters and heterogeneous environments.

### Implementation:

#### Using PyTorch:

```
from torch.nn.parallel import DistributedDataParallel as DDP

# `model` is the model we previously initialized
model = ...

# `rank` is a device number starting from 0
model = model.to(rank)
ddp_model = DDP(model, device_ids=[rank])
```

#### Using TensorFlow:

```
import tensorflow as tf

strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = Model(...)
    model.compile(...)
```

## Data Parallelism with PyTorch:

PyTorch implements data parallelism in various ways. Two of them are:

### 1. Data Parallel
```nn.DataParallel``` allows you to perform parallel training in a single machine with multiple GPUs. A major advantage is that it requires minimum code.

In [ ]:
torch.nn.DataParallel(module, device_ids=None, output_device=None, dim=0)

### 2. Distributed Data Parallel
```nn.DistributedDataParallel``` allows you to perform parallel training across multiple GPUs within multiple machines. It requires a few more extra steps to configure the training process.  

In [ ]:
torch.nn.parallel.DistributedDataParallel(module, device_ids=None, output_device=None, dim=0,
                                          broadcast_buffers=True, process_group=None, bucket_cap_mb=25,
                                          find_unused_parameters=False, check_reduction=False, gradient_as_bucket_view=False,
                                          static_graph=False, delay_all_reduce_named_params=None,
                                          param_to_hook_all_reduce=None, mixed_precision=None)

## 2) Model parallelism

In this scenario, network layers are distributed among multiple GPUs, potentially dividing the workload of individual layers. It is a technique of partioning or sharding a neural network architecture graph into subgraphs and assigning each subgraph or model shard to a different device(GPU). Each GPU handles input data flowing into specific layers, processes information across subsequent neural network layers, and then transmits the processed data to the next GPU.

Also referred to as Network Parallelism, this method segments the model into concurrent segments, executed on separate nodes but operating on the same data. The scalability here hinges on the algorithm's degree of task parallelization and is comparatively more intricate to implement than data parallelism. It might reduce communication requirements, as workers synchronize shared parameters (typically once per forward or backward-propagation step), particularly suited for GPUs within a single server connected via a high-speed bus. It is also compatible with larger models, as hardware constraints per node cease to be limiting factors.

<!-- ![model_parallelism](https://drive.google.com/uc?id=10L8Xdwo_FVVWXRryk0nHQsEWcSL_ckpM) -->
![model_parallelism](https://i.postimg.cc/280kjK1J/image.png)

### Implementation:

#### Using PyTorch:

```
import torch.nn as nn

linear1 = nn.Linear(16, 8).to('cuda:0')
linear2 = nn.Linear(8, 4).to('cuda:1')
```

#### Using TensorFlow:

```
import tensorflow as tf
from tensorflow.keras import layers

with tf.device('/GPU:0'):
    linear1 = layers.Dense(8, input_dim=16)
with tf.device('/GPU:1'):
    linear2 = layers.Dense(4, input_dim=8)
```

## Model Parallelism with PyTorch:

## Data Parallelization vs Model Parallelization

Data parallelism is more commonly used than model parallelism. While synchronous Distributed SGD can be time-consuming due to synchronization, a similar challenge arises in whole model parallelism, requiring synchronization between different parts of the neural network. However, certain models, such as inception networks, are better suited for model parallelization despite these limitations.

## 3) Pipeline Parallelization

Pipeline parallelism, aimed at sequential deep models, extends the model parallel training paradigm by forming a natural pipeline structure through a sequence of shards. Pipelining parallelism breaks the task (data and model) into a sequence of processing stages. Each stage takes the result from the previous stage as input, with results being passed downstream immendiately.

This approach aims to fill pipeline stages with execution operations, minimizing the idling inherent in traditional sequential model parallelism. Imagine a pure feedforward network; in this scenario, each shard operates as a pipeline stage, effectively transforming a model segmented across multiple GPUs into a functional multi-stage pipeline. In the context of pipeline basics, while CPU pipelining involves populating the pipeline with various CPU instructions, deep learning (DL) pipelining fills it with micro-batches akin to those employed in gradient accumulation. Consequently, pipeline parallelism integrates gradient accumulation and model parallelism, enabling independent micro-batches to traverse the shard pipeline, accumulating gradients at each stage. Once these gradients for the entire mini-batch are aggregated, they are applied to the model, showcasing a hybrid model-parallel and data-parallel method.

#### Challenges to pipeline parallelization:

Backpropagation poses a challenge in pipeline parallel training, requiring intermediate outputs for the process. A solution involves merging accumulation with checkpointing, storing activations solely at shard boundaries to reduce the need for scalability-limiting intermediate output sets. However, the bidirectional flow within the pipe, where data flows both forward during prediction and backward during backpropagation, introduces the challenge of potential collisions and necessitates a pipeline flush between prediction and backpropagation, impacting performance if not effectively managed.

Various strategies, such as augmenting micro-batch numbers or proposing alternate pipelining schedules, have been proposed to address these challenges, each with its trade-offs concerning efficiency, memory utilization, and maintaining convergence behaviors.

## 4) Hybrid Parallelism

Hybrid parallelism involves combining different parallelization methods to enhance overall performance. For instance, integrating data parallelism onto model parallelism can potentially offer both memory scalability across multiple devices and the execution speed advantages of data parallelism. However, these combined strategies come with trade-offs that must be considered during their creation. In a basic overlay hybridization, the need for multiple devices in model parallelism is multiplied by the replication demands of data parallelism. Adding task parallelism, such as in multi-model training, further multiplies the complexity. More intricate hybridizations might apply data parallelism to certain segments of a model parallel architecture while executing others sequentially. This introduces a new "search space" in the design, prompting questions about which stages to choose for data parallel replication, the impact of model parallel sharding decisions on data parallel performance, allocation of limited resources to stages, and how device interconnections and topologies influence performance.

<!-- ![hybrid_parallelization](https://drive.google.com/uc?id=1W-tQDClvUWnAivT72XBx-Fpi6w38Wd_E) -->
![hybrid_parallelization](https://i.postimg.cc/Sx5mCY7f/image.png)

## Determining Optimal Parallelism Strategies:

- **Data Parallelism**: Suitable when lacking expertise, prioritizing convergence speed, working with smaller batch sizes (e.g. < 64), and fitting the model within a single GPU. Applicable in both single-machine and multi-machine setups, effortlessly implemented within Deep Learning frameworks like PyTorch and TensorFlow, serving as an ideal starting point.

- **Model Parallelism**: Recommended for large models surpassing a single device's capacity, possessing numerous parallel branches, or aiming for high device utilization and memory efficiency. Unsuitable for multi-machine usage due to elevated communication costs; custom implementation is often necessary.

- **Pipeline Parallelism**: Utilized when the model exceeds a single device's limits but implementing Model Parallelism is overly complex. Highly effective with sequential models like CNNs and Transformers, functioning well across devices and machines due to minimal communication costs. Relatively straightforward to implement, with various existing software packages available for Deep Learning models; may require several attempts to optimally distribute the workload across devices.

- **Hybrid Parallelism**: Ideal for training significantly large models or numerous large models concurrently. Implementing Data Parallelism alongside other parallel strategies enhances convergence speed when models are distributed across workers, offering a quicker training process.